In [1]:
import surprise
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [8]:
# db = cx_Oracle.connect('c##my', 'my', 'localhost:1521/xe')
# cursor = db.cursor()
# cursor.execute('select *from PURPROD1')
# x = cursor.fetchall()
# df = pd.DataFrame(x)
# cursor.execute("select column_name from user_tab_columns where table_name = 'PURPROD1'")
# col = cursor.fetchall()
# list1 = []
# for i in col :
#     for j in i:
#         list1.append(j)
# df.columns = list1
# df.head()

,고객번호,제휴사,임의코드,통합코드,구매빈도,RANK0,RANK1
0,00001,A,A01,01,304,1,5
1,00001,A,A02,02,202,2,5
2,00001,A,A09,09,147,3,4
3,00001,A,A03,03,86,4,4
4,00001,A,A11,11,53,5,3


In [28]:
# db = cx_Oracle.connect('c##my', 'my', 'localhost:1521/xe')
# cursor = db.cursor()
# cursor.execute('select *from PURPROD2')
# x = cursor.fetchall()
# df1 = pd.DataFrame(x)
# cursor.execute("select column_name from user_tab_columns where table_name = 'PURPROD2'")
# col = cursor.fetchall()
# list1 = []
# for i in col :
#     for j in i:
#         list1.append(j)
# df1.columns = list1
# df1.head()

,임의코드,임의분류명,구매빈도
0,A01,가공식품,304
1,A02,신선식품,202
2,A03,일상용품,86
3,A07,디지털/가전,13
4,A08,가구/인테리어,20


In [3]:
df = pd.read_pickle('D:\/purprod3.pkl')
df

,고객번호,제휴사,소분류코드,임의코드,통합코드,구매빈도,RANK0,RANK1
0,00001,A,A010401,A02,02,60,1,5
1,00001,A,A010404,A02,02,59,2,5
2,00001,A,A010302,A01,01,40,3,5
3,00001,A,A040222,A09,09,29,4,5
4,00001,A,A020302,A03,03,28,5,5
...,...,...,...,...,...,...,...,...
6527915,19383,D,D030206,D01,01,1,52,1
6527916,19383,D,D010505,D03,03,1,53,1
6527917,19383,D,D010103,D03,03,1,54,1
6527918,19383,D,D040101,D01,01,1,55,1


In [11]:
df1 = pd.read_pickle('D:\/purprod2.pkl')
df1

MemoryError: 

In [4]:
reader = surprise.Reader(rating_scale = (1, 5))

data  = Dataset.load_from_df(df[['고객번호', '소분류코드', 'RANK1']], reader)

trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

algo = SVD(random_state = 0)
algo.fit(trainset)

cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4561  1.4570  1.4564  1.4567  1.4564  1.4566  0.0003  
MAE (testset)     1.1873  1.1876  1.1872  1.1876  1.1872  1.1874  0.0002  
Fit time          62.71   59.59   58.31   66.04   64.50   62.23   2.91    
Test time         24.15   24.60   20.05   20.81   19.72   21.86   2.08    


{'test_rmse': array([1.45611003, 1.45704703, 1.45644339, 1.45674227, 1.45642497]),
 'test_mae': array([1.1873317 , 1.18757636, 1.18715488, 1.18759902, 1.18723713]),
 'fit_time': (62.71316862106323,
  59.58602285385132,
  58.30770134925842,
  66.04425072669983,
  64.49675750732422),
 'test_time': (24.14819049835205,
  24.599945545196533,
  20.04795265197754,
  20.810110569000244,
  19.718740224838257)}

In [ ]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs' : [20, 40, 60, 80, 100], 'n_factors' :  [50, 100, 200]}

gs = GridSearchCV(SVD, param_grid, measures = ['rmse', 'mae'], cv = 3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

In [15]:
algo = SVD(n_epochs = 100, n_factors = 100, random_state = 0)
algo.fit(trainset)

In [16]:
def get_unpur_surprise(df, df1, 고객번호) :
    
    purs = df[df['고객번호'] == 고객번호]['임의코드'].tolist()
    
    total_pur = df['임의코드'].tolist()
    
    unpurs = [pur for pur in total_pur if pur not in purs]
    
    print('평점 매긴 제품수 : ', len(purs), '추천대상 제품수 : ', len(unpurs),
         '전체 제품수 : ', len(total_pur))
    
    return unpurs

unpurs = get_unpur_surprise(df, df, '00001')

평점 매긴 제품수 :  19 추천대상 제품수 :  95721 전체 제품수 :  346340


In [29]:
def recomm_pur_by_surprise(algo, 고객번호, unpurs, top_n = 5) :
    
    preds = [algo.predict(str(고객번호), str(임의코드)) for 임의코드 in unpurs]
    
    
    def sortkey_est(pred) :
        
        return pred.est
    
    preds.sort(key = sortkey_est, reverse = True)
    top_predictions = preds[:top_n]
    
    top_pur_ids = [str(pred.iid) for pred in top_predictions]
    top_pur_rating = [pred.est for pred in top_predictions]
    top_pur_titles = df1[df1.임의코드.isin(top_pur_ids)]['임의분류명']
    top_pur_preds = [(id, title, rating) for id, title, rating in zip(top_pur_ids, top_pur_titles, top_pur_rating)]
    
    return top_pur_preds

unpurs = get_unpur_surprise(df, df, '00001')
top_pur_preds = recomm_pur_by_surprise(algo, '00001', unpurs, top_n = 5)
print('#### Top-5 추천 제품 리스트 ####')

for top_pur in top_pur_preds:
    print(top_pur[1], ':', top_pur[2])

평점 매긴 제품수 :  19 추천대상 제품수 :  95721 전체 제품수 :  346340
#### Top-5 추천 제품 리스트 ####
교육/문화 용품 : 2.5209413543812342
교육/문화 용품 : 2.5209413543812342
교육/문화 용품 : 2.5209413543812342
교육/문화 용품 : 2.5209413543812342
교육/문화 용품 : 2.5209413543812342


In [30]:
df.RANK1.unique()

array([5, 4, 3, 2, 1], dtype=int64)